In [1]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

# Read the stock file to a dataframe
# Set the date column as the DataTimeIndex
aapl_df = pd.read_csv(
    Path("../Resources/aapl.csv"),
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True)

# Review the DataFrame
aapl_df.head()


ModuleNotFoundError: No module named 'pandas'

In [2]:
# Slice to just the `close` column
signals_df = aapl_df.loc[:,["close"]]

In [3]:
# Set the short window and long windows
short_window = 50
long_window = 100

In [4]:
# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()

# Prepopulate the `Signal` for trading
signals_df['Signal'] = 0.0

In [5]:
# Generate the trading signal 0 or 1,
# where 1 is when short-window (SMA50) is greater than the long (SMA 100)
# and 0 otherwise
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

In [6]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
signals_df.tail(10)

,close,SMA50,SMA100,Signal,Entry/Exit
date,,,,,
2019-09-06,213.260,205.2584,199.72930,1.0,0.0
2019-09-09,214.170,205.5470,199.87850,1.0,0.0
2019-09-10,216.700,205.9226,200.01420,1.0,0.0
2019-09-11,223.590,206.3634,200.21150,1.0,0.0
2019-09-12,223.085,206.7705,200.39705,1.0,0.0
2019-09-13,218.750,207.0573,200.50975,1.0,0.0
2019-09-16,219.900,207.3707,200.63715,1.0,0.0
2019-09-17,220.700,207.7843,200.79135,1.0,0.0
2019-09-18,222.770,208.2149,200.97605,1.0,0.0


In [7]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

In [8]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

In [9]:
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
signals_df['Position'] = share_size * signals_df['Signal']

In [10]:
# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

In [11]:
# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']

In [12]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum() 

In [13]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

In [14]:
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

In [15]:
# Calculate the portfolio cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

In [16]:
# Print the DataFrame
signals_df.head(150)

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2014-09-22,101.06,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2014-09-23,102.64,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
2014-09-24,101.75,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.000000,0.00000
2014-09-25,97.87,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.000000,0.00000
2014-09-26,100.75,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-21,126.91,126.5278,119.6350,1.0,0.0,500.0,0.0,63455.0,36770.0,100225.0,-0.003430,0.00225
2015-04-22,128.62,126.7058,119.7452,1.0,0.0,500.0,0.0,64310.0,36770.0,101080.0,0.008531,0.01080
2015-04-23,129.67,126.8588,119.8519,1.0,0.0,500.0,0.0,64835.0,36770.0,101605.0,0.005194,0.01605


In [17]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Apple Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)